In [4]:
#IMPORTS

import enchant  #pip install pyenchant
import numpy as np
import itertools

In [11]:
def extractUsers(text):
    
    """ 
    RECIBE: Un texto plano que puede tener cualquier cosa
    DEVUELVE: Una lista de todos los strings que estaban precedidos por un @
    EJEMPLO: 'Hola que tal soy @axel, y vivo en @capitalFederal' -----> [axel,capitalFederal]'
    """
    
    words = text.split(' ')
    
    users = []
    userToSave = ""
    for word in words:
        if len(word) != 0 and '@' in word:
            splittedWord = word.split('@')
            if len(splittedWord) > 1:
                userToSave = splittedWord[1]
            else:
                userToSave = splittedWord[0]
        if userToSave != '':
            users.append(userToSave)
    return users

In [12]:
def removeDots(word):
    """ 
    RECIBE: Una palabra cualquiera
    DEVUELVE: La parte de la palabra que no tiene ningun punto
    EJEMPLO: 'esperando...' -----> 'esperando'
    """
    
    splited = word.split('.')
    
    for chunk in splited:
        if chunk != '.':
            return chunk
    return word

In [13]:
def hasNumeric(word):
    """ 
    RECIBE: Una palabra cualquiera
    DEVUELVE: True o False dependiendo de si contiene algun numero
    EJEMPLO: 'hol4' -----> True
    """    
    
    for c in word:
        if c.isnumeric():
            return True
    return False

In [1]:
def getEnlgishWord(word):
       
    """
    RECIBE: Un texto plano que puede tener cualquier cosa
    DEVUELVE: La una tupla con la palabra mas larga en ingles que encuentre primero dentro del texto recibido
              y un True o False dependiendo de si habia o no una palabra en ingles en el texto. 
              Si no contiene ninguna palabra en ingles devuelve el texto original.
    TENER EN CUENTA: No considera como una palabra valida a las que tengan longitud 1
    EJEMPLO: 'aaaaaaaaaaENGLISHaaaaaaaaaa' -----> 'ENGLISH'
    EJEMPLO: 'aaaaaaaaaaaaaaaaaaaa' -----> 'aaaaaaaaaaaaaaaaaaaa'
    """  
    
    englishDictionary = enchant.Dict("en_US")
    
    MIN_LEN_WORDS = 2
    
    failure = False
    
    if word == None:
        failure = True
    
    elif len(word) < MIN_LEN_WORDS:
        failure = True
        
    else:
        for i in range(len(word), MIN_LEN_WORDS - 1,-1):
            for j in range(i):
                aux = word[j:i]
                if englishDictionary.check(aux) and not hasNumeric(aux) and len(aux) > 1:
                    return removeDots(aux), True
                else:
                    failure = True
    
    if failure:  
        return word, False
        

In [2]:
def mapBinaryLabel(binaryIndicator, labelsList):
    
    """ 
    RECIBE: 1 o 0
    DEVUELVE: labelsList[0] o labelsList[1] respectivamente
    """  
    
    if binaryIndicator == 1:
        return labelsList[0]
    if binaryIndicator == 0:
        return labelsList[1]
    else:
        return None

In [2]:
def generateAbsolutePath(fileName):
    
    """ 
    RECIBE: El nombre de un archivo
    DEVUELVE: El path absoluto del archivo
    EJEMPLO: 'miArchivo.txt' -----> '/home/axelmpm/DatosRepo/Data/miArchivo.txt'
    """ 
    
    return  os.path.abspath('') + '/Data/' + fileName

In [17]:
def getWordInListFromFile(fileName):
    
    """ 
    RECIBE: El nombre de un archivo donde cada linea tiene palabras
    DEVUELVE: Una lista donde cada elemento son las palabras que estaban como linea en el archivo (no toma las palabras de longitud 1)
    """ 
    
    file = open(generateAbsolutePath(fileName),'r')
    
    l = []
    
    for line in file:
        if len(line[:-1]) > 1:
            l.append(line[:-1]) #le saco el \n
    file.close()
    return l

In [3]:
def hasCertainWords(stuffWithWordsInIt,words):
    
    """ 
    RECIBE: Recibe algo que tiene palabras dentro (texto, lista de palabras, conjuntos con palabras, etc)
    DEVUELVE: Devuelve True o False dependiendo de si el el contendor contiene ALGUNA palabra en la lista de palabras
    """ 
    
    for word in words:
        if word in text:
            return True
    return False

In [11]:
def clasifyWordsRespectToEnglish(bagOfWords):
    
    """ 
    RECIBE: Un contenedor de palabras que podrian o no estar en ingles
    DEVUELVE: Una lista de palabras que estan en ingles y otra que no estaban en ingles
    EJEMPLO: {'hello..gggggg,ghghghgh,who,hola'} -------> {hello,who}, {ghghghgh,hola}
    """ 
    
    bagOfEnglishWords = []
    bagOfNonEnglishWords = []
    for word in bagOfWords:
        processedWord = getEnlgishWord(word)
        if processedWord[0] == '1':
            bagOfEnglishWords.append(processedWord[1:])
        else:
            bagOfNonEnglishWords.append(processedWord)
        
    return bagOfEnglishWords, bagOfNonEnglishWords

In [9]:
def getDataFrameOfFrecuencies(elementsLabel,elements, sort = True, sortAscending = False):
    
    """ 
    RECIBE: Una lista o secuencia de elementos y un nombre para estos
    DEVUELVE: Un dataframe con dos columnas, una de labels y otro de las frecuencias
    """ 
    
    temp_elements = list(elements.copy())
    frecs_dict = {i:temp_elements.count(i) for i in temp_elements}

    df = pd.DataFrame({elementsLabel : list(frecs_dict.keys()), 'Cantidad' : list(frecs_dict.values())})
    
    if sort:
        df.sort_values('Cantidad', ascending = sortAscending, inplace = True)
    return df

In [66]:
def getHammingSimilarity(word1,word2):
    
    """ 
    RECIBE: Dos palabras de igual longitud
    DEVUELVE: Un numero del 0 al 1 donde 1 significa que son el mismo string y 0 es que no se parecen en nada
    EJEMPLO: word1 = casa, word2 = capa ---->  0.75 (porque tienen en comun la primera c, la segunda a
             y la ultima a, osea 3/4 bien)
    """     
    
    length = len(word1)
    mismatches = 0
    
    for i in range(length):
        if word1[i] != word2[i]:
            mismatches += 1
    return 1 - mismatches/length
    

In [75]:
def generateIndexesToExtentionNChars(k,n):
    
    """ 
    RECIBE: k es la cantidad de posiciones y n es el rango de la variable
    DEVUELVE: Todas las tuplas cuyas posicion previa no tiene un valor mayor a la actual
    EJEMPLO: k = 2, n = 3 ----> {(0, 1), (0, 2), (1, 2)}  
    """
    
    indexes = list(range(n))
    l = []
    
    for i in range(k):
        l.append(indexes)
        
    allIndexCombination = set()
    
    for element in itertools.product(*l):
        allIndexCombination.add(element)
    
    indexesPos = set()

    for pos in allIndexCombination:
        
        valid = True
        for i in range(k):
            if i > 0:
                if pos[i - 1] >= pos[i]:
                    valid = False
                    break
        if valid:
            indexesPos.add(pos)
    
    return indexesPos
    

In [7]:
l = [1,2,3]
itertools.product(l,l)

In [79]:
def generateWordExtentionsIntoNChars(word,n,extentionChar):
    
    """ 
    RECIBE: Una palabra a extender, la cantidad de caracteres que debe tener la version extendida y que caracter usar al extender
    DEVUELVE: Devuelve un set de todas las posibles extensiones
    EJEMPLO: word = 'casa', n = 6, extentionChar = '.' ---->
                {'..casa','.c.asa','.ca.sa','.cas.a','.casa.','c..asa','c.a.sa','c.as.a','c.asa.','ca..sa'
                    'ca.s.a','ca.sa.','cas..a','cas.a.','casa..'}   
    """
    
    if len(word) == '':
        return [extentionChar * n]
    
    if (len(word) > n):
        return [word]
    
    if len(word) == n:
        return [word]
    
    wordLength = len(word)
    amountOfCharsToAdd = n - wordLength
    
    extentions = set()
    
    extentionCharIndexes = generateIndexesToExtentionNChars(amountOfCharsToAdd,n)
    
    for pos in extentionCharIndexes:
            
        extendedWordArray = np.empty(n, dtype=object)
        
        for i in range(amountOfCharsToAdd):
            extendedWordArray[pos[i]] = extentionChar
        
        charsPlaced = 0
        indexToTry = 0
        k = 0

        while(charsPlaced < wordLength):

            if extendedWordArray[indexToTry] == None:
                extendedWordArray[indexToTry] = word[k]
                charsPlaced += 1
                k += 1
            else:
                indexToTry += 1

        #print(extendedWordArray,indexToTry,k,charsPlaced,wordLength,pos)    
        extentions.add(''.join(list(extendedWordArray)))
    
    return extentions

In [59]:
def getSimilarity(word1,word2):
    
    """ 
    RECIBE: Dos palabras de cualquier longitud
    DEVUELVE: Un numero del 0 al 1 donde 1 significa que son el mismo string y 0 es que no se parecen en nada
    A TENER EN CUENTA: Usa la similaridad de Hamming
    EJEMPLO: word1 = casa, word2 = ca ---->  0.5 (porque tienen en comun la primera c, la segunda a, osea 2/4 bien)
    """ 
    
    EMPTY_CHARACTER = ' '
    
    if word1 + word2 == '':
        return 1
    
    if word1 + word2 == word1 or word1 + word2 == word1:
        return 0
    
    if len(word1) > len(word2):
        longest = word1
        shortest = word2
    else:
        longest = word2
        shortest = word1
        
    lenDif = len(longest) - len(shortest)
    maxSimilarity = 0
    extentions = generateWordExtentionsIntoNChars(shortest,len(longest),EMPTY_CHARACTER)
    
    for extention in extentions:
        subSimilarity = getHammingSimilarity(extention,longest)
        #print(shortest,subWord,longest,subSimilarity)
        
        if (subSimilarity >= maxSimilarity):
            maxSimilarity = subSimilarity
            
    return maxSimilarity
        

In [14]:
def isIn(element,container):

    """ 
    RECIBE: Un elemento y un contenedor de elementos
    DEVUELVE: Si el elemento esta o no en el contenedor
    """ 
    
    return element in container

In [2]:
def acortar_texto(df):
    """
    RECIBE: un df
    DEVUELVE: el mismo df con una columna con version corta de la columna 'text'
    """
    
    
    df['short_text'] = 'None'
    cant_filas = len(df.index)
    for y in range(cant_filas):
        df['short_text'][y:]= df.iloc[y][0][:15] + "..."

In [1]:
def isQuestion(text):
    '''
    RECIBE: Un string.
    DEVUELVE: True si tiene al menos una palabra en ingles seguida por al menos un signo de pregunta, sino retorna FALSE
    '''
    text = text.replace('?', ' ? ')
    englishDictionary = enchant.Dict("en_US")
    text = takeOutSpecialCaractersFromText(text, specialCharacters = [',','.','#','!','%','*','(',')','"','-','_', '$', ':', '/', '\''])
    textSplitted = text.split()
    for word in textSplitted:
        if englishDictionary.check(word):
            for j in range(textSplitted.index(word) + 1, len(textSplitted)):
                if ('?' in textSplitted[j]):
                    return True
    return False

In [2]:
def takeOutSpecialCaractersFromText(text, specialCharacters = [',','.','#','!','%','*','(',')','"','-','_', '?', '$', ':', '/', '\'']):
    '''
    RECIBE: Un string.
    DEVUELVE: El string sin caracteres especiales. Con el parametro specialCharacters se puede decidir que caracteres sacar.
    '''
    for char in specialCharacters:
        text = text.replace(char, ' ')
    return text